# Scraping Script

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import requests
import os
import time
import random

In [ ]:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0",
]

In [ ]:
def scrape(search_query, num_images=100):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")

    service = Service("D:\\chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=chrome_options)

    driver.get(f"https://www.bing.com/images/search?q={search_query}&FORM=HDRSC2")
    image_selector = "img.mimg"

    if not os.path.exists(search_query):
        os.makedirs(search_query)

    images = []
    downloaded = 0
    last_height = driver.execute_script("return document.body.scrollHeight")

    while downloaded < num_images:
        new_images = driver.find_elements(By.CSS_SELECTOR, image_selector)
        for img in new_images[len(images):]:
            img_url = img.get_attribute("src")
            if img_url and img_url.startswith("http") and downloaded < num_images:
                try:
                    img_data = requests.get(img_url, timeout=10).content
                    with open(f"{search_query}/{search_query}_{downloaded + 1}.jpg", "wb") as img_file:
                        img_file.write(img_data)
                    print(f"Downloaded: {img_url}")
                    downloaded += 1
                except Exception as e:
                    print(f"Failed to download {img_url}: {e}")

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(1, 3))

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("No more images to load or CAPTCHA detected.")
            break
        last_height = new_height

        if "captcha" in driver.page_source.lower():
            print("CAPTCHA detected! Pausing to avoid detection.")
            time.sleep(10)
            driver.refresh()
            time.sleep(10)

    driver.quit()


In [ ]:
scrape("kantong plastik putih", num_images=200)